In [246]:
import json
import pandas as pd 
import glob
import os
import re
from bs4 import BeautifulSoup 
from IPython.display import display, HTML
import pickle

In [28]:
## Read in consolidated annotations
annotations={}
annofiles=glob.glob("Data/opp115-parsed-annotation-1.0/*.json")
print len(annofiles)
policyFiles=glob.glob("../Data/OPP-115/sanitized_policies/*.html")
print len(policyFiles)

115
115


###  Read in the annotations and original policies

In [40]:
def readAnno(filelist):
  annotations={}
  for filename in filelist:
    website=re.sub(".json", '', os.path.basename(filename))
    with open(filename, "r") as f:
      annotations[website]=json.load(f)
    f.close()
  return annotations

def readPolicies(filelist):
  soups={}
  for filename in filelist:
    base=os.path.basename(filename).split("_")[1]
    website=re.sub(".html", '', base)
    soups[website]=BeautifulSoup(open(filename, "r").read(), 'html.parser')
  return soups

In [248]:
with open("Data/parsed-annotation-0.5.pk", 'rb') as f:
  annotations=pickle.load(f)

In [251]:
annotations["playstation.com"]["Data Retention"]['Retention Purpose']

,endIndexInSegment,section,selectedText,startIndexInSegment,value
0,153,18,promotional purpose through one of our websites,106,Marketing
1,201,18,or to make a purchase from the PlayStation Shop,154,Perform service
2,226,22,Email addresses collected from consumers durin...,0,Perform service
3,200,24,so that we may assist these customers with cur...,124,Perform service
4,159,40,necessary to fulfill the purposes outlined in ...,102,Other


In [51]:
policySoups=readPolicies(policyFiles)

In [72]:
## make sure that the website names are the same in both dictinoaries
test1=filter(lambda website: website in annotations.keys(), policySoups.keys())
test2=filter(lambda website: website in policySoups.keys(), annotations.keys())
print all(test1)
print all(test2)

True
True


## Get a list of tokens from all of the policies, with frequencies

In [56]:
def extractTexts(soups):
  policyTexts={}
  for website in soups:
    policyTexts[website]=soups[website].get_text()
  return policyTexts

In [57]:
## get texts from soup objects
policyTexts=extractTexts(policySoups)

In [160]:
## get the training and validation set 
import random
websites=policyTexts.keys()
seed=124
random.seed(seed)
trainWebsites=random.sample(websites, 105)
valWebstes=[web for web in websites if web not in trainWebsites]

trainTexts={web:policyTexts[web] for web in trainWebsites}

In [161]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import itertools
import string
engstop=set(stopwords.words("english"))


In [144]:
print string.punctuation

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [245]:
from collections import Counter
def cleantext(rawtext, remove):
  # remove punctuations
  cleaned3=remove.sub("", rawtext.lower())
  return cleaned3

def getTokens(rawtext, mode, stopwords):
  punctuations = re.compile('[%s]' % re.escape(string.punctuation))
  if mode=="unigram":
    cleaned=cleantext(rawtext, punctuations)
    tokenlist=word_tokenize(cleaned)
    tokensNoStop=[token for token in tokenlist if token not in stopwords]
    return list(set(tokensNoStop))
  
  if mode=="bigram":
    sentenceList=sent_tokenize(rawtext)
    sentenceClean=[cleantext(sentence, punctuations) for sentence in sentenceList]
    unigramLists=[word_tokenize(sentence) for sentence in sentenceClean]
    bigramLists=[zip(uniList, uniList[1::]) for uniList in unigramLists]
    bigramTotal=[bigram for bigramlist in bigramLists for bigram in bigramlist]
    
    return list(set(bigramTotal))

In [149]:
def getAllTokens(textDict, mode, stopwords):
  allTokens=[]
  if mode=="unigram":
    for website in textDict:
      allTokens.extend(getTokens(textDict[website], "unigram", stopwords)) 
    
  elif mode=="bigram":
    for website in textDict:
      allTokens.extend(getTokens(textDict[website], "bigram", stopwords))
    return list(set(allTokens))                   
           

In [162]:
%%time
trainUnigrams=getAllTokens(trainTexts, "unigram", engstop)
trainBigrams=getAllTokens(trainTexts, "bigram", engstop)

CPU times: user 3.28 s, sys: 0 ns, total: 3.28 s
Wall time: 3.26 s


In [241]:
hondaSoup=policySoups["honda.com"]
hondaText=hondaSoup.get_text()
hondaText.split("|||")[52][250:360]

u" name, email address, and your friend's email address. This information is collected and used only in a manner"

In [196]:
psSoup=policySoups['playstation.com']
psSoup.get_text()

u' Privacy Policy    Last Revised: April, 2011  |||Sony Computer Entertainment America LLC ("SCEA") is committed to respecting the privacy rights of all visitors to our websites. This privacy policy is intended to provide you with information on how we collect, use and store the information that you provide to us through our websites so that you can make appropriate choices for sharing information with us. If you have any questions, complaints or comments regarding our online or offline privacy policies, please contact SCEA\'s Consumer Services Hotline at 1-800-345-7669.  |||This Privacy Statement and the certification seal located to your right confirms that SCEA is a valid licensee and participating member in the Entertainment Software Rating Board\'s Privacy Online Program: ESRB Privacy Online. To protect your privacy to the maximum extent possible, we have undertaken this privacy initiative and our websites have been reviewed and certified by ESRB Privacy Online to meet established

In [225]:
annotations['playstation.com']['First Party Collection/Use']["Collection Mode"].keys()

[u'selectedText', u'startIndexInSegment', u'endIndexInSegment', u'value']

In [218]:
#print(annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"].keys())
#print annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]["selectedText"]

print annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]["startIndexInSegment"][0]
#print annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]['endIndexInSegment'][0]

textinfo=pd.DataFrame({"selectedText":annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]["selectedText"],\
                      "startIndexInSegment": annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]["startIndexInSegment"],\
                      "endIndexInSegment": annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]['endIndexInSegment'],\
                      "value": annotations['playstation.com']['First Party Collection/Use']["Personal Information Type"]['value']})


textinfo=textinfo[["selectedText", "value", "startIndexInSegment", "endIndexInSegment"]]

344


In [232]:
textinfo.loc[1,"selectedText"]
print textinfo.head()

                                        selectedText  \
0                                        information   
1  PlayStationNetwork In parts of North America a...   
2                 personally identifying information   
3  may include the collection of date of birth, n...   
4                     mailing address, email address   

                          value  startIndexInSegment  endIndexInSegment  
0                   Unspecified                  344                355  
1                         Other                   41                680  
2  Generic personal information                  483                517  
3                   Demographic                  804                854  
4                       Contact                  855                885  


In [220]:
psSections=psSoup.get_text().split("|||")

In [233]:
psSections[6][41:680]

u'ionNetwork   In parts of North America and South America, Sony Network Entertainment America Inc. ("SNEA") operates Sony Online Services, a network of online games, movies, music, other media and content and communication services. PlayStation Network ("PSN") is one of these Sony Online Services. With a Sony Online Services or Sony Entertainment Network account, users can purchase goods and services from SNEA through Sony Online Services and may have the opportunity to participate in various network community activities. Users can register for and log into a Sony Entertainment Network account via us.playstation.com. Collection and '

In [222]:
## Label the sentences in a policy as relevant or irrelevant
## 
# Topic: "Personal Information Type"

PS=trainTexts["playstation.com"]
PSsentences=trainTexts